# *FaunaScope*

**Autor:** Juan Carlos Ramírez Gil  
**Fecha:** 7 de noviembre de 2025  
**Proyecto:** AVES_MAMIFEROS 
**Software:** Python 3.14 + Jupyter Notebook  
---


# Introduccion

Este código corresponde a u análisis estadístico de la diversidad par aves y mamíferos en la parte alta de cuenca del rio zulia en el departamento de norte de Santander como parte de una prestacion de servicios profesionales en Biologia (Fauna), el siguiente código integra Python desde visual code  para crear un código de análisis reproducible y moderno que permita ahorrar el trabajo de análisis en proyectos futuros.

## Crear repositorio en GitHub para guardar el proyecto

El proyecto sera guardado en un repositorio de github para poder darle trazabilidad al mismo y compartirlo libremente la siguiente linea de codigo permite guardarlo en automatico ejecutando #### auto_commit.py


In [ ]:
import subprocess
from datetime import datetime

def ejecutar_comando(comando):
    """Ejecuta un comando del sistema y muestra la salida en consola."""
    try:
        resultado = subprocess.run(
            comando,
            check=True,
            text=True,
            capture_output=True
        )
        print(resultado.stdout)
    except subprocess.CalledProcessError as e:
        print(f"⚠️ Error ejecutando comando: {comando}")
        print(e.stderr)

def auto_commit():
    """Agrega, commitea y sube los cambios automáticamente al repositorio."""
    
    # 🕒 1️⃣ Generar mensaje con fecha y hora
    fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    mensaje = f"Auto-commit: actualización {fecha}"

    print("🔍 Verificando estado del repositorio...")
    ejecutar_comando(["git", "status"])

    print("➕ Agregando todos los cambios al área de preparación...")
    ejecutar_comando(["git", "add", "."])

    print(f"💾 Realizando commit con mensaje: '{mensaje}'")
    ejecutar_comando(["git", "commit", "-m", mensaje])

    print("🚀 Subiendo cambios al repositorio remoto (rama master)...")
    ejecutar_comando(["git", "push", "origin", "master"])


# Analisis de datos
Hay que recordar que se trabajara desde python, utilizando pandas y leyendo archivos xlsx, para esto se aplicar un paquete especifico que lea el documento

In [ ]:
#--------------## Cargar librerias necesarias------------------------------

# Si no las tienes instaladas, ejecuta esta celda una vez:
# Salir del interprete con: exit()

# !pip install pandas numpy matplotlib tabulate openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import openpyxl

#--------------## Leer archivo ------------------------------
# Ruta del archivo
ruta = r"D:\CORPONOR 2025\Backet\python_Proyect\data\POF_ZULIA_2025_BD_AVES_MAMIFEROS.xlsx"

# Leer el archivo Excel
Registros = pd.read_excel(ruta)

# Mostrar las primeras filas
print("📄 Primeras filas del archivo:")
print(Registros.head())

# Mostrar nombres de las columnas
print("\n📋 Columnas del DataFrame:")
print(Registros.columns)


# Metodologia


A continuación, se describen los análisis realizados sobre la información obtenida durante el muestreo de fauna.


## Esfuerzo de Muestreo
El esfuerzo de muestreo (E) se estimó de acuerdo con la metodología empleada y las características del grupo taxonómico evaluado, considerando la relación entre el número de unidades de muestreo, el tiempo efectivo de muestreo y, cuando aplicó, el número de observadores o dispositivos utilizados. En términos generales, el esfuerzo se calculó multiplicando el número de unidades de muestreo (trampas, redes, cámaras, transectos o puntos de observación) por el tiempo de exposición o duración de cada sesión de muestreo (horas, días o noches), ajustado al número de personas o equipos activos en campo. Esta aproximación permite estandarizar la intensidad del muestreo entre diferentes métodos y grupos biológicos, facilitando la comparación de resultados y la estimación del éxito de muestreo (Em), definido como la razón entre el número total de registros obtenidos y el esfuerzo total aplicado para cada técnica.

##### En primera instancia se revisa el contenido de las columas a utilizar en el analisis

In [ ]:
#--------------## Leer archivo y revisar columnas------------------------------
# Ruta del archivo
ruta = r"D:\CORPONOR 2025\Backet\python_Proyect\data\POF_ZULIA_2025_BD_AVES_MAMIFEROS.xlsx"

# Leer el archivo Excel
Registros = pd.read_excel(ruta)

# Mostrar las primeras filas
print("📄 Primeras filas del archivo:")
print(Registros.head())

# Mostrar nombres de las columnas
print("\n📋 Columnas del DataFrame:")
print(Registros.columns)

# Mostrar valores únicos de algunas columnas
for col in ["METODOLOGIA", "METODO", "ID", "Gremio"]:
    if col in Registros.columns:
        print(f"\n🔹 Valores únicos en '{col}':")
        print(Registros[col].unique())
    else:
        print(f"\n⚠️ La columna '{col}' no existe en el DataFrame.")

#### Crear tabla de esfuerzo de muestreo

In [ ]:
#--------------## Esfuerzo de Muestreo------------------------------

import pandas as pd
from tabulate import tabulate

# --- Copiar dataframe base ---
df = Registros.copy()

# --- Normalizar texto ---
df['METODO'] = df['METODO'].astype(str).str.strip()
df['COBERTURA'] = df['COBERTURA'].astype(str).str.strip()
df['ID'] = df['ID'].astype(str).str.strip()

# --- Diccionario de abreviaciones de coberturas ---
abreviaciones_cobertura = {
    'Bosque De Galería Y Ripario': 'Bgr',
    'Bosque De Galeria Y Ripario': 'Bgr',
    'Bosque Denso Alto De Tierra Firme': 'Bda',
    'Bosque Denso Bajo De Tierra Firme': 'Bdb',
    'Bosque Fragmentado Con Vegetación Secundaria': 'Bfvs',
    'Bosque Fragmentado Con Vegetacion Secundaria': 'Bfvs',
    'Sin Dato': 'NA'
}

# --- Aplicar reemplazos (con control de mayúsculas y tildes) ---
df['COBERTURA'] = df['COBERTURA'].apply(
    lambda x: abreviaciones_cobertura.get(x.strip().title(), x)
)

# --- Reemplazar vacíos y nulos por 'Sin dato' ---
df['METODO'] = df['METODO'].replace('', 'Sin dato').fillna('Sin dato')
df['COBERTURA'] = df['COBERTURA'].replace('', 'Sin dato').fillna('Sin dato')


# --- Validar que exista la nueva columna de horas ---
if 'Hora_Hombre' not in df.columns:
    raise ValueError('❌ La columna Hora_Hombre no existe en el dataframe Registros.')

# --- Calcular totales de individuos (sin perder registros) ---
individuos = (
    df.groupby(['METODO', 'COBERTURA'], dropna=False, as_index=False)['INDIVIDUOS']
      .sum(min_count=1)
)

# --- Calcular esfuerzo total único por ID ---
# (Evitamos duplicar horas si un ID aparece varias veces)
esfuerzo_unico = df[['ID', 'METODO', 'COBERTURA', 'Hora_Hombre']].drop_duplicates()

# --- Calcular esfuerzo total (solo una vez por ID) ---
esfuerzo = (
    esfuerzo_unico.groupby(['METODO', 'COBERTURA'], dropna=False, as_index=False)['Hora_Hombre']
    .sum(min_count=1)
    .rename(columns={'Hora_Hombre': 'Esfuerzo_horas'})
)


# --- Unir tablas ---
tabla = individuos.merge(esfuerzo, on=['METODO', 'COBERTURA'], how='outer')
tabla['Exito_captura'] = tabla['INDIVIDUOS'] / tabla['Esfuerzo_horas']

# --- Calcular totales por método ---
totales = tabla.groupby('METODO', as_index=False).agg({
    'INDIVIDUOS': 'sum',
    'Esfuerzo_horas': 'sum'
})
totales['Exito_captura'] = totales['INDIVIDUOS'] / totales['Esfuerzo_horas']
totales['COBERTURA'] = 'Total'

# --- Unir con la tabla principal ---
tabla_final = pd.concat([tabla, totales], ignore_index=True)

# --- Reestructurar para salida ---
tabla_melt = pd.melt(
    tabla_final,
    id_vars=['METODO', 'COBERTURA'],
    value_vars=['INDIVIDUOS', 'Esfuerzo_horas', 'Exito_captura'],
    var_name='Indice',
    value_name='Valor'
)

# --- Cambiar nombres de los índices ---
tabla_melt['Indice'] = tabla_melt['Indice'].replace({
    'INDIVIDUOS': 'Número de individuos',
    'Esfuerzo_horas': 'Esfuerzo captura (horas-hombre)',
    'Exito_captura': 'Éxito de captura (individuos/horas-hombre)'
})


# --- Orden lógico de los índices ---
orden_indices = [
    'Número de individuos',
    'Esfuerzo captura (horas-hombre)',
    'Éxito de captura (individuos/horas-hombre)'
]
tabla_melt['Indice'] = pd.Categorical(tabla_melt['Indice'], categories=orden_indices, ordered=True)

# --- Renombrar columna ---
tabla_melt = tabla_melt.rename(columns={'METODO': 'Metodologia'})

# --- Orden personalizado de metodologias ---
orden_metodologia = [
    'Transecto',
    'Punto de observacion',
    'Red de niebla',
    'Camara Trampa',
    'Informacion Secundaria'
]

tabla_melt['Metodologia'] = pd.Categorical(tabla_melt['Metodologia'], categories=orden_metodologia, ordered=True)

# --- Orden personalizado de Metodologias ---
orden_COBERTURA = [
    'Bgr',
    'Bfvs',
    'Bda',
    'Bdb',
    'Total'
]

tabla_melt['COBERTURA'] = pd.Categorical(tabla_melt['COBERTURA'], categories=orden_COBERTURA, ordered=True)

# --- Pivotar ---
tabla_pivot = tabla_melt.pivot_table(
    index=['Metodologia', 'Indice'],
    columns='COBERTURA',
    values='Valor',
    aggfunc='first'
).reset_index()

# --- Redondear ---
tabla_pivot = tabla_pivot.round(3)

# --- Mostrar resumen en consola ---
print(tabulate(tabla_pivot, headers='keys', tablefmt='fancy_grid', floatfmt='.3f'))


# --- Exportar a Excel a una ruta específica ---
import os

# Definir la ruta exacta donde guardar el archivo
output_path = r"D:\CORPONOR 2025\Backet\python_Proyect\Resultados"
output_file = os.path.join(output_path, "Esfuerzo_Muestreo.xlsx")

# Exportar el DataFrame a Excel
tabla_pivot.to_excel(output_file, index=False)

# Confirmar la ubicación del archivo guardado
print(f"✅ Archivo exportado correctamente en:\n{output_file}")

#### Dar Formato a la tabla exportada

In [ ]:
#---------------Dar formato a archivo generato o tabla---------------


from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from openpyxl.utils import get_column_letter
import os

# --- Nombre del archivo a formatear ---
output_file = r"D:\CORPONOR 2025\Backet\python_Proyect\Resultados\Esfuerzo_Muestreo.xlsx"

# --- Verificar que el archivo existe ---
if not os.path.exists(output_file):
    raise FileNotFoundError(f"⚠️ No se encontró el archivo: {output_file}")

# --- Cargar el archivo ---
wb = load_workbook(output_file)
ws = wb.active

# --- Estilos base ---
header_fill = PatternFill(start_color='BFD8B8', end_color='BFD8B8', fill_type='solid')
header_font = Font(bold=True, color='000000', name='Calibri')
center_align = Alignment(horizontal='center', vertical='center', wrap_text=True)

# --- Bordes finos para toda la tabla ---
thin_border = Border(
    left=Side(style='thin', color='000000'),
    right=Side(style='thin', color='000000'),
    top=Side(style='thin', color='000000'),
    bottom=Side(style='thin', color='000000')
)

# --- Aplicar formato y reemplazar vacíos ---
for row in ws.iter_rows():
    for cell in row:
        # Reemplazar vacíos o None por guion
        if cell.value is None or str(cell.value).strip() == '':
            cell.value = '-'
        # Aplicar formato general
        cell.border = thin_border
        cell.alignment = center_align

# --- Aplicar formato al encabezado ---
for cell in ws[1]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = center_align

# --- Ajustar ancho de columnas automáticamente ---
for col in ws.columns:
    max_length = 0
    column = get_column_letter(col[0].column)
    for cell in col:
        if cell.value:
            length = len(str(cell.value))
            if length > max_length:
                max_length = length
    ws.column_dimensions[column].width = max_length + 3

# --- Ajustar altura de filas automáticamente ---
for row in ws.iter_rows():
    max_height = 15
    for cell in row:
        if cell.value and "\n" in str(cell.value):
            lines = str(cell.value).count('\n') + 1
            if lines > 1:
                max_height = 15 * lines
    ws.row_dimensions[cell.row].height = max_height

# --- Guardar cambios ---
wb.save(output_file)
print(f'📘 Archivo formateado con éxito:\n{output_file}')


#### Mostrar archivo creado

In [1]:
import pandas as pd
from IPython.display import display, HTML

# --- Leer el archivo Excel ---
# 👇 Usa una cadena RAW (r"...") para evitar errores con las barras invertidas
tabla = pd.read_excel(r"D:\CORPONOR 2025\Backet\python_Proyect\Resultados\Esfuerzo_Muestreo.xlsx")

# --- Mostrar tabla con desplazamiento vertical ---
display(HTML(f"""
<h3>Vista del archivo <code>Esfuerzo_Muestreo.xlsx</code></h3>
<div style="
    height: 400px;
    overflow-y: scroll;
    border: 1px solid #ccc;
    padding: 8px;
    font-size: 14px;
">
{tabla.to_html(index=False)}
</div>
"""))



Metodologia,Indice,Bgr,Bfvs,Bda,Bdb,Total
Transecto,Número de individuos,160,243,1683.000,713.000,2799.000
Transecto,Esfuerzo captura (horas-hombre),12,30,108.000,78.000,228.000
Transecto,Éxito de captura (individuos/horas-hombre),13.333,8.1,15.583,9.141,12.276
Punto de observacion,Número de individuos,-,-,11.000,5.000,16.000
Punto de observacion,Esfuerzo captura (horas-hombre),-,-,6.000,2.000,8.000
Punto de observacion,Éxito de captura (individuos/horas-hombre),-,-,1.833,2.500,2.000
Red de niebla,Número de individuos,-,1,10.000,2.000,13.000
Red de niebla,Esfuerzo captura (horas-hombre),-,2,9.000,2.000,13.000
Red de niebla,Éxito de captura (individuos/horas-hombre),-,0.5,1.111,1.000,1.000
Camara Trampa,Número de individuos,-,-,17.000,68.000,85.000
